# Cleaning NBA Player Data

The goal of this Notebook is to clean up the NBA player data that I previously extracted from the Basketball Reference website (www.basketball-reference.com).

In [1]:
# Import the necessary libraries for cleaning the NBA player data
import sys
import pandas as pd
import numpy as np

In [2]:
# load the file from my Google Drive
players_df = pd.read_csv('/content/drive/MyDrive/NBA_players_data.csv')

## Initial Investigation and Cleaning

Let's print the dataset to get an idea of what we're going to have to do to it.

In [3]:
players_df

,Player,From,To,Pos,Ht,Wt,Birth Date,Colleges,G,G.1,...,25,26,27,28,29,30,31,32,33,34
0,Alaa Abdelnaby,1991,1995,F-C,6-10,240.0,"June 24, 1968",Duke,256,256,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Zaid Abdul-Aziz,1969,1978,C-F,6-9,235.0,"April 7, 1946",Iowa State,505,505,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Kareem Abdul-Jabbar*,1970,1989,C,7-2,225.0,"April 16, 1947",UCLA,1560,1560,...,4x BLK Champ,6x NBA Champ,15x All-NBA,11x All-Defensive,1969-70 All-Rookie,1969-70 ROY,2x Finals MVP,6x MVP,NBA 75th Anniv. Team,NaN
3,Mahmoud Abdul-Rauf,1991,2001,G,6-1,162.0,"March 9, 1969",LSU,586,586,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Tariq Abdul-Wahad,1998,2003,F,6-6,223.0,"November 3, 1974","Michigan, San Jose State",236,236,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5018,Ante Žižić,2018,2020,F-C,6-10,266.0,"January 4, 1997",NaN,113,113,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5019,Jim Zoet,1983,1983,C,7-1,240.0,"December 20, 1953",Kent State University,7,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5020,Bill Zopf,1971,1971,G,6-1,170.0,"June 7, 1948",Duquesne,53,53,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5021,Ivica Zubac,2017,2022,C,7-0,240.0,"March 18, 1997",NaN,360,360,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


As we can see from the print out of the dataset, there seem to be some missing values, missing columns, and values in the incorrect column. 

## Career Accolades Adjustment


The first issue we will address is sorting through the career accolades and placing them in the correct column. When we scraped that data, if a player had any accolades, it would place the accolades in the first available column. This presents us with a problem because we could have all sorts of different accolades in a single column. We need to make uniform columns for each possible accolade a player could have.

We will first save the size of the DataFrame in order to loop over it later. 

In [4]:
size = players_df.shape
print(size[1])

35


Now that we have the size of the DataFrame saved, we will apply some initial cleaning to make things easier. In it's infancy, the NBA was originally the Basketball Association of America (BAA). The BAA ran for 3 years and then absorbed the National Basketball League (NBL) to form the modern NBA. Players who played in the BAA may have accolades with 'BAA' in the title, so we will change these to read 'NBA' since they are the same league and it will make cleaning easier.

In [5]:
players_df.replace('(BAA)', 'NBA', regex=True, inplace=True)

Now we will create the columns for each of the possible career accolades a player could have. Then we will run a for loop for the columns that are currently storing the career accolades, looking for specific key phrases that will help indicate what the accolade is. After identifying the accolade, then we will reassign it to the correct column.

In [6]:
players_df[['All Star', 'All NBA', 'All ABA', 'All Rookie', 'All Defensive', 
         'BLK Champ', 'STL Champ', 'TRB Champ', 'AST Champ',
         'Scoring Champ', 'Most Improved', 'Sixth Man', 'DPOY',
         'ROY', 'AS MVP', 'CF MVP', 'Finals MVP', 'MVP',
         'NBA Championships', 'ABA Championships', 'NBA 75 Team', 'ABA All-Time Team', 'HOF']] = 0
for i in range(0, size[0]):
  for j in range(21, size[1]):
    if pd.isnull(players_df[str(j)].iloc[i]):
      continue
    elif players_df[str(j)].str.contains('All Star').iloc[i] == True:
      players_df.iat[i, size[1]] = players_df[str(j)].iloc[i]
    elif players_df[str(j)].str.contains('All-NBA').iloc[i] == True:
      players_df.iat[i, size[1]+1] = players_df[str(j)].iloc[i]
    elif players_df[str(j)].str.contains('All-ABA').iloc[i] == True:
      players_df.iat[i, size[1]+2] = players_df[str(j)].iloc[i]
    elif players_df[str(j)].str.contains('All-Rookie').iloc[i] == True:
      players_df.iat[i, size[1]+3] = 1
    elif players_df[str(j)].str.contains('All-Defensive').iloc[i] == True:
      players_df.iat[i, size[1]+4] = players_df[str(j)].iloc[i]
    elif players_df[str(j)].str.contains('BLK Champ').iloc[i] == True:
      players_df.iat[i, size[1]+5] = players_df[str(j)].iloc[i]
    elif players_df[str(j)].str.contains('STL Champ').iloc[i] == True:
      players_df.iat[i, size[1]+6] = players_df[str(j)].iloc[i]
    elif players_df[str(j)].str.contains('TRB Champ').iloc[i] == True:
      players_df.iat[i, size[1]+7] = players_df[str(j)].iloc[i]
    elif players_df[str(j)].str.contains('AST Champ').iloc[i] == True:
      players_df.iat[i, size[1]+8] = players_df[str(j)].iloc[i]
    elif players_df[str(j)].str.contains('Scoring Champ').iloc[i] == True:
      players_df.iat[i, size[1]+9] = players_df[str(j)].iloc[i]
    elif players_df[str(j)].str.contains('Most Improved').iloc[i] == True:
      players_df.iat[i, size[1]+10] = players_df[str(j)].iloc[i]
    elif players_df[str(j)].str.contains('Sixth Man').iloc[i] == True:
      players_df.iat[i, size[1]+11] = players_df[str(j)].iloc[i]
    elif players_df[str(j)].str.contains('Def. POY').iloc[i] == True:
      players_df.iat[i, size[1]+12] = players_df[str(j)].iloc[i]
    elif players_df[str(j)].str.contains('ROY').iloc[i] == True:
      players_df.iat[i, size[1]+13] = 1
    elif players_df[str(j)].str.contains('AS MVP').iloc[i] == True:
      players_df.iat[i, size[1]+14] = players_df[str(j)].iloc[i]
    elif players_df[str(j)].str.contains('CF MVP').iloc[i] == True:
      players_df.iat[i, size[1]+15] = players_df[str(j)].iloc[i]
    elif players_df[str(j)].str.contains('Finals MVP').iloc[i] == True:
      players_df.iat[i, size[1]+16] = players_df[str(j)].iloc[i]
    elif players_df[str(j)].str.contains('MVP').iloc[i] == True:
      players_df.iat[i, size[1]+17] = players_df[str(j)].iloc[i]
    elif players_df[str(j)].str.contains('NBA Champ').iloc[i] == True:
      players_df.iat[i, size[1]+18] = players_df[str(j)].iloc[i]
    elif players_df[str(j)].str.contains('ABA Champ').iloc[i] == True:
      players_df.iat[i, size[1]+19] = players_df[str(j)].iloc[i]
    elif players_df[str(j)].str.contains('Anniv. Team').iloc[i] == True:
      players_df.iat[i, size[1]+20] = 1
    elif players_df[str(j)].str.contains('All-Time').iloc[i] == True:
      players_df.iat[i, size[1]+21] = 1
    elif players_df[str(j)].str.contains('Hall of Fame').iloc[i] == True:
      players_df.iat[i, size[1]+22] = 1
    else:
      continue

Next, we will do the same thing for the 'WS' column, as some accolades were saved in that column as well.

In [7]:
for i in range(0, size[0]):
  if pd.isnull(players_df['WS'].iloc[i]):
    continue
  elif players_df['WS'].str.contains('All Star').iloc[i] == True:
    players_df.iat[i, size[1]] = players_df['WS'].iloc[i]
  elif players_df['WS'].str.contains('All-NBA').iloc[i] == True:
    players_df.iat[i, size[1]+1] = players_df['WS'].iloc[i]
  elif players_df['WS'].str.contains('All-ABA').iloc[i] == True:
    players_df.iat[i, size[1]+2] = players_df['WS'].iloc[i]
  elif players_df['WS'].str.contains('All-Rookie').iloc[i] == True:
    players_df.iat[i, size[1]+3] = 1
  elif players_df['WS'].str.contains('All-Defensive').iloc[i] == True:
    players_df.iat[i, size[1]+4] = players_df['WS'].iloc[i]
  elif players_df['WS'].str.contains('BLK Champ').iloc[i] == True:
    players_df.iat[i, size[1]+5] = players_df['WS'].iloc[i]
  elif players_df['WS'].str.contains('STL Champ').iloc[i] == True:
    players_df.iat[i, size[1]+6] = players_df['WS'].iloc[i]
  elif players_df['WS'].str.contains('TRB Champ').iloc[i] == True:
    players_df.iat[i, size[1]+7] = players_df['WS'].iloc[i]
  elif players_df['WS'].str.contains('AST Champ').iloc[i] == True:
    players_df.iat[i, size[1]+8] = players_df['WS'].iloc[i]
  elif players_df['WS'].str.contains('Scoring Champ').iloc[i] == True:
    players_df.iat[i, size[1]+9] = players_df['WS'].iloc[i]
  elif players_df['WS'].str.contains('Most Improved').iloc[i] == True:
    players_df.iat[i, size[1]+10] = players_df['WS'].iloc[i]
  elif players_df['WS'].str.contains('Sixth Man').iloc[i] == True:
    players_df.iat[i, size[1]+11] = players_df['WS'].iloc[i]
  elif players_df['WS'].str.contains('Def. POY').iloc[i] == True:
    players_df.iat[i, size[1]+12] = players_df['WS'].iloc[i]
  elif players_df['WS'].str.contains('ROY').iloc[i] == True:
    players_df.iat[i, size[1]+13] = 1
  elif players_df['WS'].str.contains('AS MVP').iloc[i] == True:
    players_df.iat[i, size[1]+14] = players_df['WS'].iloc[i]
  elif players_df['WS'].str.contains('CF MVP').iloc[i] == True:
    players_df.iat[i, size[1]+15] = players_df['WS'].iloc[i]
  elif players_df['WS'].str.contains('Finals MVP').iloc[i] == True:
    players_df.iat[i, size[1]+16] = players_df['WS'].iloc[i]
  elif players_df['WS'].str.contains('MVP').iloc[i] == True:
    players_df.iat[i, size[1]+17] = players_df['WS'].iloc[i]
  elif players_df['WS'].str.contains('NBA Champ').iloc[i] == True:
    players_df.iat[i, size[1]+18] = players_df['WS'].iloc[i]
  elif players_df['WS'].str.contains('ABA Champ').iloc[i] == True:
    players_df.iat[i, size[1]+19] = players_df['WS'].iloc[i]
  elif players_df['WS'].str.contains('Anniv. Team').iloc[i] == True:
    players_df.iat[i, size[1]+20] = 1
  elif players_df['WS'].str.contains('All-Time').iloc[i] == True:
    players_df.iat[i, size[1]+21] = 1
  elif players_df['WS'].str.contains('Hall of Fame').iloc[i] == True:
    players_df.iat[i, size[1]+22] = 1
  else:
    continue

In [8]:
players_df.to_csv('/content/drive/MyDrive/NBA_players_test.csv', index=False)

In [9]:
players_df = pd.read_csv('/content/drive/MyDrive/NBA_players_test.csv')

After placing each accolade in the correct column, we need to clean them by only saving a numeric value. So, we will strip away any strings or characters so that we are left with only integers. Then we need to change values that are for only a specific year/season to a value of 1.

In [10]:
for column in players_df[['All Star', 'All ABA', 'All Rookie', 'All Defensive', 
         'BLK Champ', 'STL Champ', 'TRB Champ', 'AST Champ',
         'Scoring Champ', 'Most Improved', 'Sixth Man',
         'ROY', 'AS MVP', 'Finals MVP', 'MVP', 'NBA 75 Team', 'ABA All-Time Team', 'HOF']]:
  if players_df[column].dtype == 'int64':
    continue
  elif players_df[column].dtype == 'float64' or players_df[column].dtype == 'float':
    continue
  else:
    players_df[column] = players_df[column].str.replace('-', ' ')
    players_df[column] = players_df[column].str.replace(column, '')
    players_df[column] = players_df[column].str.replace(' ', '')
    players_df[column] = players_df[column].str.replace('x', '')
    players_df[column] = players_df[column].astype('int')
    players_df[column].values[players_df[column].values > 1000] = 1

Some of the columns have special characters or issues that need to be addressed separately, so we will address those columns below. 

In [ ]:
# Clean Defensive Player of the Year column
players_df['DPOY'] = players_df['DPOY'].str.replace('-', ' ')
players_df['DPOY'] = players_df['DPOY'].str.replace('Def. POY', '')
players_df['DPOY'] = players_df['DPOY'].str.replace(' ', '')
players_df['DPOY'] = players_df['DPOY'].str.replace('x', '')
players_df['DPOY'] = players_df['DPOY'].astype('int')
players_df['DPOY'].values[players_df['DPOY'].values > 1000] = 1

In [12]:
# Clean Conference Finals MVP column
players_df['CF MVP'] = players_df['CF MVP'].str.replace(' ECF MVP', '')
players_df['CF MVP'] = players_df['CF MVP'].str.replace(' WCF MVP', '')
players_df['CF MVP'] = players_df['CF MVP'].str.replace('-', '')
players_df['CF MVP'] = players_df['CF MVP'].str.replace('x', '')
players_df['CF MVP'] = players_df['CF MVP'].astype('int')
players_df['CF MVP'].values[players_df['CF MVP'].values > 1000] = 1

In [13]:
# Clean NBA Championships column
players_df['NBA Championships'] = players_df['NBA Championships'].str.replace('-', ' ')
players_df['NBA Championships'] = players_df['NBA Championships'].str.replace('NBA Champ', '')
players_df['NBA Championships'] = players_df['NBA Championships'].str.replace('NBA', '')
players_df['NBA Championships'] = players_df['NBA Championships'].str.replace('/', '')
players_df['NBA Championships'] = players_df['NBA Championships'].str.replace(' ', '')
players_df['NBA Championships'] = players_df['NBA Championships'].str.replace('x', '')
players_df['NBA Championships'] = players_df['NBA Championships'].astype('int')
players_df['NBA Championships'].values[players_df['NBA Championships'].values > 1000] = 1

In [14]:
# Clean All NBA column
players_df['All NBA'] = players_df['All NBA'].str.replace('-', ' ')
players_df['All NBA'] = players_df['All NBA'].str.replace('All NBA', '')
players_df['All NBA'] = players_df['All NBA'].str.replace('NBA', '')
players_df['All NBA'] = players_df['All NBA'].str.replace('/', '')
players_df['All NBA'] = players_df['All NBA'].str.replace(' ', '')
players_df['All NBA'] = players_df['All NBA'].str.replace('x', '')
players_df['All NBA'] = players_df['All NBA'].astype('int')
players_df['All NBA'].values[players_df['All NBA'].values > 1000] = 1

In [15]:
# Clean ABA Championships column
players_df['ABA Championships'] = players_df['ABA Championships'].str.replace('-', ' ')
players_df['ABA Championships'] = players_df['ABA Championships'].str.replace('ABA Champ', '')
players_df['ABA Championships'] = players_df['ABA Championships'].str.replace(' ', '')
players_df['ABA Championships'] = players_df['ABA Championships'].str.replace('x', '')
players_df['ABA Championships'] = players_df['ABA Championships'].astype('int')
players_df['ABA Championships'].values[players_df['ABA Championships'].values > 1000] = 1

After the final changes to the accolades, let's look at the columns that we have.

In [16]:
players_df.columns

Index(['Player', 'From', 'To', 'Pos', 'Ht', 'Wt', 'Birth Date', 'Colleges',
       'G', 'G.1', 'PTS', 'TRB', 'AST', 'FG%', 'FG%.1', 'FG3%', 'FT%', 'eFG%',
       'PER', 'PER.1', 'WS', '21', '22', '23', '24', '25', '26', '27', '28',
       '29', '30', '31', '32', '33', '34', 'All Star', 'All NBA', 'All ABA',
       'All Rookie', 'All Defensive', 'BLK Champ', 'STL Champ', 'TRB Champ',
       'AST Champ', 'Scoring Champ', 'Most Improved', 'Sixth Man', 'DPOY',
       'ROY', 'AS MVP', 'CF MVP', 'Finals MVP', 'MVP', 'NBA Championships',
       'ABA Championships', 'NBA 75 Team', 'ABA All-Time Team', 'HOF'],
      dtype='object')

Now that we have assigned the accolades to their correct columns, we can go ahead and delete the columns that are numbered. We will also get rid of 'Birth Date' and 'Colleges' as they will have no impact on whether or not a player enters the Hall of Fame. We can also get rid of any duplicate columns as well. 

In [17]:
# Remove duplicate columns in the DataFrame
players_df.drop(['Birth Date', 'Colleges', 'G.1', 'FG%.1', 'PER.1', '21', '22', '23', '24',
                 '25', '26', '27', '28', '29', '30', '31', '32', '33', '34'], axis=1, inplace=True)
players_df.replace('-', np.nan, inplace=True)

In [18]:
players_df.columns

Index(['Player', 'From', 'To', 'Pos', 'Ht', 'Wt', 'G', 'PTS', 'TRB', 'AST',
       'FG%', 'FG3%', 'FT%', 'eFG%', 'PER', 'WS', 'All Star', 'All NBA',
       'All ABA', 'All Rookie', 'All Defensive', 'BLK Champ', 'STL Champ',
       'TRB Champ', 'AST Champ', 'Scoring Champ', 'Most Improved', 'Sixth Man',
       'DPOY', 'ROY', 'AS MVP', 'CF MVP', 'Finals MVP', 'MVP',
       'NBA Championships', 'ABA Championships', 'NBA 75 Team',
       'ABA All-Time Team', 'HOF'],
      dtype='object')

We can then replace any accolades in the 'WS' column with a 'NaN', which we will address later.

In [19]:
players_df['WS'].replace('[A-Za-z]', np.nan, regex=True, inplace=True)
errors = players_df['WS'].str.contains('[A-Za-z]')
errors.value_counts()

False    3895
Name: WS, dtype: int64

After removing these columns, we can take a look at how many missing values there are for each column remaining.

In [20]:
missing_data = players_df.isnull()
for column in missing_data.columns.values.tolist():
    print(column)
    print (missing_data[column].value_counts())
    print("")

Player
False    5023
Name: Player, dtype: int64

From
False    5023
Name: From, dtype: int64

To
False    5023
Name: To, dtype: int64

Pos
False    5023
Name: Pos, dtype: int64

Ht
False    5023
Name: Ht, dtype: int64

Wt
False    5018
True        5
Name: Wt, dtype: int64

G
False    5023
Name: G, dtype: int64

PTS
False    5023
Name: PTS, dtype: int64

TRB
False    4731
True      292
Name: TRB, dtype: int64

AST
False    5023
Name: AST, dtype: int64

FG%
False    4989
True       34
Name: FG%, dtype: int64

FG3%
False    4463
True      560
Name: FG3%, dtype: int64

FT%
False    4482
True      541
Name: FT%, dtype: int64

eFG%
False    4650
True      373
Name: eFG%, dtype: int64

PER
False    5019
True        4
Name: PER, dtype: int64

WS
False    3895
True     1128
Name: WS, dtype: int64

All Star
False    5023
Name: All Star, dtype: int64

All NBA
False    5023
Name: All NBA, dtype: int64

All ABA
False    5023
Name: All ABA, dtype: int64

All Rookie
False    5023
Name: All Rookie, dt

We can see that there are quite a few columns still missing data, which we will need to address.

## Correction of Non-Uniform Player Pages

The biggest change that needs to be made to the data at this stage, is the correction of where the career statistics has been placed in the DataFrame. There is some data that has been misplaced or duplicated. The main cause of this is due to the non-uniformity of the players webpage's where the career statistics are downloaded. Some players have all of the columns seen in the DataFrame, others are missing 'FG3%' and 'eFG%', so we must address this before making any more changes to the data.

After close examination of the data, we can tell if the data has been placed incorrectly if there are some duplicate values and there is a 'NaN' in the 'WS' column. For these players, the FT% was added to the FG3% column, the PER was added to both the FT% and eFG% columns, and the WS was in the PER column. Needless to say, we have to correct this and move the correct data to the correct columns, while making sure the FG3% and eFG% columns have a 'NaN', which we will handle later. 

Let's take a look at the data type for each column, to make sure they are what we want them to be.

In [21]:
players_df.dtypes

Player                object
From                   int64
To                     int64
Pos                   object
Ht                    object
Wt                   float64
G                      int64
PTS                  float64
TRB                   object
AST                  float64
FG%                   object
FG3%                  object
FT%                   object
eFG%                  object
PER                   object
WS                    object
All Star               int64
All NBA                int64
All ABA                int64
All Rookie             int64
All Defensive          int64
BLK Champ              int64
STL Champ              int64
TRB Champ              int64
AST Champ              int64
Scoring Champ          int64
Most Improved          int64
Sixth Man              int64
DPOY                   int64
ROY                    int64
AS MVP                 int64
CF MVP                 int64
Finals MVP             int64
MVP                    int64
NBA Championsh

There are a few columns that are type 'object' that we need to be type 'float'. So, we will change them to be the correct data type before we rearrange the career stats.

In [22]:
players_df[['TRB', 'FG%', 'FG3%', 'FT%', 'eFG%', 'PER', 'WS']] = players_df[['TRB', 'FG%', 'FG3%', 'FT%', 'eFG%', 'PER', 'WS']].astype('float')

In [23]:
for i in range(0,len(players_df)):
  if np.isnan(players_df['WS'].iloc[i]):
    players_df['WS'].iloc[i] = players_df['PER'].iloc[i] # the 'WS' column value is actually the value in the 'PER' column
    players_df['PER'].iloc[i] = players_df['eFG%'].iloc[i] # the 'PER' column value is actually the value in the 'eFG%' column
    players_df['eFG%'].iloc[i] = np.nan # 'eFG%' doesn't have a value, so we assign it 'NaN'
    players_df['FT%'].iloc[i] = players_df['FG3%'].iloc[i] # the 'FT%' column value is actually the value in the 'FG3%' column
    players_df['FG3%'].iloc[i] = np.nan # 'FG3%' doesn't have a value, so we assign it 'NaN'

players_df.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,Player,From,To,Pos,Ht,Wt,G,PTS,TRB,AST,...,ROY,AS MVP,CF MVP,Finals MVP,MVP,NBA Championships,ABA Championships,NBA 75 Team,ABA All-Time Team,HOF
0,Alaa Abdelnaby,1991,1995,F-C,6-10,240.0,256,5.7,3.3,0.3,...,0,0,0,0,0,0,0,0,0,0
1,Zaid Abdul-Aziz,1969,1978,C-F,6-9,235.0,505,9.0,8.0,1.2,...,0,0,0,0,0,0,0,0,0,0
2,Kareem Abdul-Jabbar*,1970,1989,C,7-2,225.0,1560,24.6,11.2,3.6,...,1,0,0,2,6,6,0,1,0,1
3,Mahmoud Abdul-Rauf,1991,2001,G,6-1,162.0,586,14.6,1.9,3.5,...,0,0,0,0,0,0,0,0,0,0
4,Tariq Abdul-Wahad,1998,2003,F,6-6,223.0,236,7.8,3.3,1.1,...,0,0,0,0,0,0,0,0,0,0


As we can see now, all the right values are in the right place.

## Combining Positions that are the Same

Now let's see the different positions players have played.

In [24]:
players_df['Pos'].value_counts()

G      1795
F      1441
C       532
F-C     413
G-F     397
C-F     226
F-G     219
Name: Pos, dtype: int64

There are a couple values that are the same position but they have values flip-flopped. Specifically, 'F-C' and 'C-F' are the same position, and 'F-G' and 'G-F' are also the same position. So, we will condense these 2 instances where there are 2 values for the same position down into 1 single position. 

In [ ]:
for i in range(0,len(players_df)):
  if players_df['Pos'].iloc[i] == 'C-F': # combine 'C-F' and 'F-C' to be one position
    players_df['Pos'].iloc[i] = 'F-C'
  elif players_df['Pos'].iloc[i] == 'F-G': # combine 'G-F' and 'F-G' to be one position
    players_df['Pos'].iloc[i] = 'G-F'
  else:
    players_df['Pos'].iloc[i] = players_df['Pos'].iloc[i]

In [26]:
players_df['Pos'].value_counts()

G      1795
F      1441
F-C     639
G-F     616
C       532
Name: Pos, dtype: int64

Now, we see that there are 5 distinct position types. Creating these specific position values is important for the next step of the data cleaning, filling in the remaining missing values, as the position a player plays has a big impact on different statistics. 

## Filling in Missing Values

After moving values to the correct columns, there are still a couple of columns that have missing data. Getting rid of all of these rows would remove a large portion of the dataset, which we don't want. We also don't want to fill each missing value with the mean of the column it is in, as basketball statistics depend heavily on the position of the player. Therefore, we will find the average for each position in the columns that are missing data, and then fill in the value for the position of that player. In other words, if a player plays the 'G' position and is missing a value for 'FG3%', we will fill that missing value with the mean of 'FG3%' for the position 'G'.

In [27]:
tfg_avg = players_df.groupby('Pos', as_index=False)['FG3%'].mean() # calculate the mean of 'FG3%' for each position
eff_avg = players_df.groupby('Pos', as_index=False)['eFG%'].mean() # calculate the mean of 'eFG%' for each position
fg_avg = players_df.groupby('Pos', as_index=False)['FG%'].mean() # calculate the mean of 'FG%' for each position
ft_avg = players_df.groupby('Pos', as_index=False)['FT%'].mean() # calculate the mean of 'FT%' for each position
wt_avg = players_df.groupby('Pos', as_index=False)['Wt'].mean() # calculate the mean of 'Wt' for each position
per_avg = players_df.groupby('Pos', as_index=False)['PER'].mean() # calculate the mean of 'PER' for each position
reb_avg = players_df.groupby('Pos', as_index=False)['TRB'].mean() # calculate the mean of 'TRB' for each position
ws_avg = players_df.groupby('Pos', as_index=False)['WS'].mean() # calculate the mean of 'WS' for each position
print(tfg_avg)
print('--------------')
print(eff_avg)
print('--------------')
print(fg_avg)
print('--------------')
print(ft_avg)
print('--------------')
print(wt_avg)
print('--------------')
print(per_avg)
print('--------------')
print(reb_avg)
print('--------------')
print(ws_avg)

   Pos       FG3%
0    C  14.200000
1    F  23.351274
2  F-C  17.749140
3    G  27.939540
4  G-F  28.615854
--------------
   Pos       eFG%
0    C  45.932955
1    F  45.279222
2  F-C  48.450000
3    G  44.077183
4  G-F  47.109198
--------------
   Pos        FG%
0    C  43.746958
1    F  40.973739
2  F-C  44.524765
3    G  38.627762
4  G-F  40.867427
--------------
   Pos        FT%
0    C  62.537800
1    F  67.610542
2  F-C  67.880787
3    G  73.740982
4  G-F  72.892105
--------------
   Pos          Wt
0    C  245.073585
1    F  218.525694
2  F-C  226.541471
3    G  188.160625
4  G-F  200.689935
--------------
   Pos        PER
0    C  11.148805
1    F  10.922345
2  F-C  13.603578
3    G  10.286178
4  G-F  12.460149
--------------
   Pos       TRB
0    C  4.081102
1    F  3.047080
2  F-C  5.489580
3    G  1.812368
4  G-F  3.133333
--------------
   Pos         WS
0    C  14.614474
1    F   9.265625
2  F-C  23.708294
3    G  10.894318
4  G-F  20.471104


As we can see from the outputs, the 5 different positions have widely different means for most of the statistics. For example, the mean 'FT%' for 'G' is 73.7%, but the mean for 'C' is 62.5%, over 10% less. 

Now, that we have the means for each value in 'Pos', we will loop through our dataset and replace missing values with the player's corresponding position's mean.

In [ ]:
# Loop for 'FG3%'
for i in range(0,len(players_df)):
  if np.isnan(players_df['FG3%'].iloc[i]):
    val = tfg_avg[tfg_avg['Pos'] == players_df['Pos'].iloc[i]]['FG3%'] # find the players position and the mean value for that position
    players_df['FG3%'].iloc[i] = val.round(decimals=1)
  else:
    continue
# Loop for 'eFG%'
for i in range(0,len(players_df)):
  if np.isnan(players_df['eFG%'].iloc[i]):
    val = eff_avg[eff_avg['Pos'] == players_df['Pos'].iloc[i]]['eFG%'] # find the players position and the mean value for that position
    players_df['eFG%'].iloc[i] = val.round(decimals=1)
  else:
    continue
# Loop for 'FG%'
for i in range(0,len(players_df)):
  if np.isnan(players_df['FG%'].iloc[i]):
    val = fg_avg[fg_avg['Pos'] == players_df['Pos'].iloc[i]]['FG%'] # find the players position and the mean value for that position
    players_df['FG%'].iloc[i] = val.round(decimals=1)
  else:
    continue
# Loop for 'FT%'
for i in range(0,len(players_df)):
  if np.isnan(players_df['FT%'].iloc[i]):
    val = ft_avg[ft_avg['Pos'] == players_df['Pos'].iloc[i]]['FT%'] # find the players position and the mean value for that position
    players_df['FT%'].iloc[i] = val.round(decimals=1)
  else:
    continue
# Loop for 'Wt'
for i in range(0,len(players_df)):
  if np.isnan(players_df['Wt'].iloc[i]):
    val = wt_avg[wt_avg['Pos'] == players_df['Pos'].iloc[i]]['Wt'] # find the players position and the mean value for that position
    players_df['Wt'].iloc[i] = val.round(decimals=1)
  else:
    continue
# Loop for 'PER'
for i in range(0,len(players_df)):
  if np.isnan(players_df['PER'].iloc[i]):
    val = per_avg[per_avg['Pos'] == players_df['Pos'].iloc[i]]['PER'] # find the players position and the mean value for that position
    players_df['PER'].iloc[i] = val.round(decimals=1)
  else:
    continue
# Loop for 'TRB'
for i in range(0,len(players_df)):
  if np.isnan(players_df['TRB'].iloc[i]):
    val = reb_avg[reb_avg['Pos'] == players_df['Pos'].iloc[i]]['TRB'] # find the players position and the mean value for that position
    players_df['TRB'].iloc[i] = val.round(decimals=1)
  else:
    continue
# Loop for 'WS'
for i in range(0,len(players_df)):
  if np.isnan(players_df['WS'].iloc[i]):
    val = ws_avg[ws_avg['Pos'] == players_df['Pos'].iloc[i]]['WS'] # find the players position and the mean value for that position
    players_df['WS'].iloc[i] = val.round(decimals=1)
  else:
    continue

Now let's look at the updated dataset.

In [29]:
players_df.head()

,Player,From,To,Pos,Ht,Wt,G,PTS,TRB,AST,...,ROY,AS MVP,CF MVP,Finals MVP,MVP,NBA Championships,ABA Championships,NBA 75 Team,ABA All-Time Team,HOF
0,Alaa Abdelnaby,1991,1995,F-C,6-10,240.0,256,5.7,3.3,0.3,...,0,0,0,0,0,0,0,0,0,0
1,Zaid Abdul-Aziz,1969,1978,F-C,6-9,235.0,505,9.0,8.0,1.2,...,0,0,0,0,0,0,0,0,0,0
2,Kareem Abdul-Jabbar*,1970,1989,C,7-2,225.0,1560,24.6,11.2,3.6,...,1,0,0,2,6,6,0,1,0,1
3,Mahmoud Abdul-Rauf,1991,2001,G,6-1,162.0,586,14.6,1.9,3.5,...,0,0,0,0,0,0,0,0,0,0
4,Tariq Abdul-Wahad,1998,2003,F,6-6,223.0,236,7.8,3.3,1.1,...,0,0,0,0,0,0,0,0,0,0


## Final Changes

We want to know how many years each player was in the NBA for, as this is more valuable than knowing the values for 'From' and 'To'.

In [30]:
# Add a new column for the number of years each player was in the NBA
#players_df['Years'] = players_df['To'] - players_df['From'] + 1
players_df.insert(3, 'Years', players_df['To'] - players_df['From'] + 1)
players_df.head()

,Player,From,To,Years,Pos,Ht,Wt,G,PTS,TRB,...,ROY,AS MVP,CF MVP,Finals MVP,MVP,NBA Championships,ABA Championships,NBA 75 Team,ABA All-Time Team,HOF
0,Alaa Abdelnaby,1991,1995,5,F-C,6-10,240.0,256,5.7,3.3,...,0,0,0,0,0,0,0,0,0,0
1,Zaid Abdul-Aziz,1969,1978,10,F-C,6-9,235.0,505,9.0,8.0,...,0,0,0,0,0,0,0,0,0,0
2,Kareem Abdul-Jabbar*,1970,1989,20,C,7-2,225.0,1560,24.6,11.2,...,1,0,0,2,6,6,0,1,0,1
3,Mahmoud Abdul-Rauf,1991,2001,11,G,6-1,162.0,586,14.6,1.9,...,0,0,0,0,0,0,0,0,0,0
4,Tariq Abdul-Wahad,1998,2003,6,F,6-6,223.0,236,7.8,3.3,...,0,0,0,0,0,0,0,0,0,0


It is worth noting that some players took years off in between when they started and finished their career, so the value for years may not be entirely accurate for every single player.

The ABA and NBA were 2 distinct leagues, however, for a player we really only care about the total number of championships, and not which league it was in. So, we will combine the values for NBA and ABA Championships into one column and one value. 

In [31]:
players_df.insert(37, 'Championships', players_df['ABA Championships'] + players_df['NBA Championships'])
players_df.drop(['NBA Championships', 'ABA Championships'], axis=1, inplace=True)
players_df.head()

,Player,From,To,Years,Pos,Ht,Wt,G,PTS,TRB,...,DPOY,ROY,AS MVP,CF MVP,Finals MVP,MVP,Championships,NBA 75 Team,ABA All-Time Team,HOF
0,Alaa Abdelnaby,1991,1995,5,F-C,6-10,240.0,256,5.7,3.3,...,0,0,0,0,0,0,0,0,0,0
1,Zaid Abdul-Aziz,1969,1978,10,F-C,6-9,235.0,505,9.0,8.0,...,0,0,0,0,0,0,0,0,0,0
2,Kareem Abdul-Jabbar*,1970,1989,20,C,7-2,225.0,1560,24.6,11.2,...,0,1,0,0,2,6,6,1,0,1
3,Mahmoud Abdul-Rauf,1991,2001,11,G,6-1,162.0,586,14.6,1.9,...,0,0,0,0,0,0,0,0,0,0
4,Tariq Abdul-Wahad,1998,2003,6,F,6-6,223.0,236,7.8,3.3,...,0,0,0,0,0,0,0,0,0,0


Now that we've cleaned the data and updated any missing values, let's look at some statistics for the dataset.

In [32]:
players_df.describe()

,From,To,Years,Wt,G,PTS,TRB,AST,FG%,FG3%,...,DPOY,ROY,AS MVP,CF MVP,Finals MVP,MVP,Championships,NBA 75 Team,ABA All-Time Team,HOF
count,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,...,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000
mean,1988.400956,1992.587896,5.186940,209.318754,270.059327,6.411328,3.035537,1.416564,40.867768,23.953036,...,0.007963,0.017121,0.016723,0.000398,0.010551,0.015329,0.180968,0.015130,0.005973,0.018714
std,22.471835,23.180842,4.509547,26.015055,310.280656,4.764540,2.244231,1.358773,10.720956,13.200200,...,0.132134,0.129736,0.172014,0.019952,0.158674,0.207304,0.664048,0.122084,0.077059,0.135526
min,1947.000000,1947.000000,1.000000,114.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1971.000000,1974.500000,1.000000,190.000000,32.000000,2.900000,1.500000,0.500000,36.700000,17.700000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1990.000000,1996.000000,3.000000,210.000000,128.000000,5.200000,2.500000,1.000000,42.300000,25.700000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2008.000000,2014.000000,8.000000,225.000000,447.000000,8.700000,4.000000,1.900000,46.400000,32.100000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2022.000000,2022.000000,23.000000,360.000000,1611.000000,30.100000,22.900000,11.200000,100.000000,100.000000,...,4.000000,1.000000,4.000000,1.000000,6.000000,6.000000,11.000000,1.000000,1.000000,1.000000


From these statistics we can see some interesting things. The average NBA player averages 6.4 points, 3 rebounds, and 1.4 assists per game, and also plays in an average of 270 games over 5.2 years in  the NBA. 

## Splitting Current/Recent and Former Players

The last thing we want to do is split the dataset into 2 different datasets, one for current NBA players and recent NBA players (who may become Hall of Famers soon) and one for former NBA players. The dataset of former players will be used to train and test a model to predict whether a player will be a Hall of Famer or not. Once the model is trained, we will use the current/recent players dataset to make predictions. We set the criteria for recent players as having a value in 'To' of 4 years before the current season. We do this because the NBA only allows players who have been retired for 4 years or more to be elligible for the Hall of Fame.

In [33]:
recent_df = players_df[players_df['To'] >= max(players_df['To']) - 4] # current players
former_df = players_df[players_df['To'] < max(players_df['To']) - 4] # former players

We then add a column to the former players dataset, 'HOF', which tells us if each inidividual player is in the Hall of Fame or not. If a player has an asterisk (*) next to their name, they are in the Hall of Fame. In the new 'HOF' column, a value of 0 denotes that they are not in the Hall of Fame, and a value of 1 denotes that they are in the Hall of Fame. 

In [ ]:
former_df['HOF'] = 0
for i in range(0,len(former_df)):
  if former_df['Player'].str.contains('\*').iloc[i] == True:
    former_df['HOF'].iloc[i] = 1
  else:
    former_df['HOF'].iloc[i] = 0

In [35]:
former_df['HOF'].value_counts()

0    3859
1     163
Name: HOF, dtype: int64

In [36]:
len(recent_df)

1001

We can see that, of the former NBA players, only 163 out of 4,022 are in the Hall of Fame, about 4%. Since there are 1,001 current and recent players, we should expect about 40 of them to become Hall of Famers.

The final thing to do is to reset the index for the current/recent and former player datasets.

In [37]:
recent_df.reset_index(drop=True, inplace=True)

In [38]:
former_df.reset_index(drop=True, inplace=True)

Finally, let's save all 3 of the datasets to csv files to use for data exploration and analysis.

In [39]:
players_df.to_csv('/content/drive/MyDrive/NBA_players_clean.csv', index=False)
former_df.to_csv('/content/drive/MyDrive/former_players.csv', index=False)
recent_df.to_csv('/content/drive/MyDrive/recent_players.csv', index=False)